In [2]:
from ctypes import *
import math
import random
import os
import numpy as np
from darknet import *

ModuleNotFoundError: No module named 'darknet'

In [ ]:
# Training process was done by YOLOv2 (AlexeyAB)
# The code is available at https://github.com/AlexeyAB/darknet

In [ ]:
# preprocessing
# cara goblok2annya kiki

In [ ]:
# The code is available at https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/
def non_max_suppression_fast(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
 
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
 
    # initialize the list of picked indexes	
    pick = []
 
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
 
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
 
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
 
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
 
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
 
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
 
    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick].astype("int")

In [5]:
test_dir = '/home/dominikus/study/deep-learning-visual-recognition/HW3/dataset/SVHN_HW/test'
test_img = os.listdir(test_dir)

# order ascendingly
test_img = [int(i.strip('.png')) for i in test_img]
test_img.sort()
test_img = [str(i)+'.png' for i in test_img]

In [ ]:
# prediction
weight = '/home/dominikus/darknet/backup/yolov2_last.weights'
# order matter, be careful
# im_path = '/home/dominikus/darknet/data/test_svhn.txt'

In [ ]:
# threshold 0.2, no nms
detection_02_nonms = []
for idx, i in enumerate(test_img):
    # im_path = dir_+i
    im_path = test_dir+'/'+i
    detection_02_nonms.append(performDetect(imagePath=im_path, thresh= 0.2, configPath = "./cfg/yolov2.cfg", weightPath = "./backup/yolov2_last.weights", metaPath= "./data/svhn.data", showImage= False, makeImageOnly = False, initOnly= False))
    if idx % 500 == 0:
        print(idx)

In [ ]:
# threshold 0.2, nms 0.5

In [ ]:
# threshold 0.2, nms 0.6

In [ ]:
# threshold 0.2, nms 0.7

In [ ]:
# threshold 0.3, nms 0.5

In [ ]:
# threshold 0.3, nms 0.6

In [ ]:
# threshold 0.3, nms 0.7

In [ ]:
# threshold 0.3, no nms

In [ ]:
# convert submission